In [2]:
#coding=utf-8
!python --version
!pwd
!ls -l test.dat.202508261856
!ls -l train.dat.202508261856
!wc -l test.dat.202508261856
!wc -l train.dat.202508261856
!head -n 1 train.dat.202508261856 | awk -F ',' '{print "train NF",NF}'
!head -n 1 test.dat.202508261856  | awk -F ',' '{print "test NF",NF}'
!ls

Python 3.7.5
/d/GH/GitWorkSpace/bank_model_competiton/data/202508261856
-rw-r--r-- 1 chenchen 197121 4908650 Aug 26 19:14 test.dat.202508261856
-rw-r--r-- 1 chenchen 197121 13447289 Aug 26 19:14 train.dat.202508261856
20055 test.dat.202508261856
53481 train.dat.202508261856
train NF 37
test NF 36
model_stacking_v3.ipynb
process_v3.ipynb
test.dat.202508261856
train.dat.202508261856


In [3]:
# -*- coding: utf-8 -*-
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
import pandas as pd
import xgboost as xgb
import datetime
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
import sys
import os
import time 

dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level"
dummy_fea = dummy_fea_str.split(',')

suffix = os.path.split(os.getcwd())[-1]

train_dat_path = "train.dat.%s" % suffix 
test_dat_path = "test.dat.%s"  % suffix 
print(train_dat_path, test_dat_path, suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

train.dat.202508261856 test.dat.202508261856 202508261856
process time :  2025-08-26 19:15:48


In [4]:

def xgb_feature(X_train, y_train, X_test, y_test=None):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'seed': 1111,
              'silent':1
              }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=800)
    predict = model.predict(dvali)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)

def xgb_feature2(X_train, y_train, X_test, y_test=None):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.015,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'seed': 11,
              'silent':1
              }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=1200)
    predict = model.predict(dvali)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)

def xgb_feature3(X_train, y_train, X_test, y_test=None):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.01,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'seed': 1,
              'silent':1
              }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=2000)
    predict = model.predict(dvali)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)


def et_model(X_train, y_train, X_test, y_test=None):
    model = ExtraTreesClassifier(max_features = 'log2', n_estimators = 1000 , n_jobs = -1).fit(X_train,y_train)
    return model.predict_proba(X_test)[:,1]

def gbdt_model(X_train, y_train, X_test, y_test=None):
    model = GradientBoostingClassifier(learning_rate = 0.02, max_features = 0.7, n_estimators = 700 , max_depth = 5).fit(X_train,y_train)
    predict = model.predict_proba(X_test)[:,1]
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)

def logistic_model(X_train, y_train, X_test, y_test=None):
    model = LogisticRegression(penalty = 'l2').fit(X_train,y_train)
    return model.predict_proba(X_test)[:,1]

def lgb_feature(X_train, y_train, X_test, y_test=None):
    lgb_train = lgb.Dataset(X_train, y_train,categorical_feature={'sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound','account_grade','industry'})
    lgb_test = lgb.Dataset(X_test,categorical_feature={'sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound','account_grade','industry'})
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':'auc',
        'num_leaves': 25,
        'learning_rate': 0.01,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'min_data_in_leaf':5,
        'max_bin':200,
        'verbose': 0,
    }
    gbm = lgb.train(params,lgb_train,num_boost_round=2000)
    predict = gbm.predict(X_test)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)


## 验证v1

In [5]:
from sklearn.model_selection import train_test_split

dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level,interest_rate_cut"
dummy_fea = dummy_fea_str.split(',')

sample_data = pd.read_csv(train_dat_path, engine = 'python');
sample_data.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)

sample_data = sample_data.fillna(0)
dummy_df = pd.get_dummies(sample_data.loc[:,dummy_fea])
print('dummy_df \n',dummy_df)

sample_data_concat = pd.concat([sample_data, dummy_df], axis=1)
sample_data_concat =sample_data_concat.fillna(0)
sample_data_concat.replace([np.inf, -np.inf], 0, inplace=True)

vaild_sample_data = sample_data_concat.drop(dummy_fea, axis=1)
print('valid sample data :\n ',vaild_sample_data.columns)
print('valid sample data :\n ',vaild_sample_data)

sample_data_target = vaild_sample_data['label'].values
sample_data_x = vaild_sample_data.drop(['label'],axis=1)

dummy_df 
           id  title  career  zip_code  residence  syndicated  installment  \
0          0      9     0.0    221373          1           0            1   
1          1      8    10.0    311681          0           0            0   
2          2      8     7.0    271562          1           0            0   
3          3      7     2.0    522083          0           0            1   
4          4      8     3.0    101026          1           0            0   
...      ...    ...     ...       ...        ...         ...          ...   
53475  53475      2     2.0    603000          1           0            0   
53476  53476      0    10.0    601702          1           0            0   
53477  53477      2    10.0    602808          1           0            0   
53478  53478      0    10.0    602102          2           0            0   
53479  53479      0    10.0    602207          1           0            0   

       level_A0  level_A1  level_A2  ...  interest_rate_cut_inte

# 模型验证

In [6]:
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

X_train, X_test, y_train, y_test = train_test_split(sample_data_x, sample_data_target, test_size=0.2, random_state=10, shuffle=True)

# # logistic model
# predict_result = logistic_model(X_train, y_train, X_test, None)
# print('logistic_model valid auc :', roc_auc_score(y_test, predict_result))

# gbdt model
predict_result = gbdt_model(X_train, y_train, X_test, None)
print('gbdt_model valid auc : ', roc_auc_score(y_test, predict_result))

print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

gbdt_model valid auc :  0.6545047144173186


ValueError: incomplete format

## 预测

In [55]:
# from sklearn.model_selection import train_test_split

# dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level"
# dummy_fea = dummy_fea_str.split(',')

# sample_data = pd.read_csv('train.dat', engine = 'python');
# sample_data = sample_data.fillna(0)
# dummy_df = pd.get_dummies(sample_data.loc[:,dummy_fea])
# print('dummy_df \n',dummy_df)

# sample_data_concat = pd.concat([sample_data, dummy_df], axis=1)
# sample_data_concat =sample_data_concat.fillna(0)
# vaild_sample_data = sample_data_concat.drop(dummy_fea, axis=1)
# print('valid sample data :\n ',vaild_sample_data.columns)

# sample_data_target = vaild_sample_data['label'].values
# sample_data_x = vaild_sample_data.drop(['label'],axis=1)

# X_train, X_test, y_train, y_test = train_test_split(sample_data_x, sample_data_target, test_size=0.2, random_state=10, shuffle=True)

train_data = pd.read_csv(train_dat_path, engine = 'python');
train_data = train_data.fillna(0)

test_data = pd.read_csv(test_dat_path, engine = 'python');
test_data = test_data.fillna(0)

train_test_data = pd.concat([train_data, test_data], axis=0, ignore_index = True)
train_test_data.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)

train_test_data = train_test_data.fillna(0)
train_test_data.replace([np.inf, -np.inf], 0, inplace=True)

dummy_df = pd.get_dummies(train_test_data.loc[:,dummy_fea])
train_test_data = pd.concat([train_test_data, dummy_df],axis=1)
train_test_data = train_test_data.drop(dummy_fea, axis=1)

train_train = train_test_data.iloc[:train_data.shape[0],:]
test_test = train_test_data.iloc[train_data.shape[0]:,:]

y_train = train_train['label'].values
X_train = train_train.drop(['label'],axis=1)
X_test = test_test.drop(['label'],axis=1)

# gbdt model
predict_result = gbdt_model(X_train, y_train, X_test, None)
# print('gbdt_model valid auc : ', roc_auc_score(y_test, predict_result))

len(predict_result)

20054

In [56]:
X_test.reset_index(inplace=True)
X_test

,index,loan,term,interest_rate,issue_time,record_time,history_time,total_accounts,balance_accounts,balance_limit,...,interest_rate_cut_interest_rate_18,interest_rate_cut_interest_rate_19,interest_rate_cut_interest_rate_2,interest_rate_cut_interest_rate_3,interest_rate_cut_interest_rate_4,interest_rate_cut_interest_rate_5,interest_rate_cut_interest_rate_6,interest_rate_cut_interest_rate_7,interest_rate_cut_interest_rate_8,interest_rate_cut_interest_rate_9
0,53480,5950,36,14.10,1054600001,1054600080,795400642,17.0,9.0,10100.0,...,0,0,0,0,0,0,1,0,0,0
1,53481,15350,36,5.25,1172880047,1172882338,756003297,17.0,9.0,58600.0,...,0,0,0,0,0,0,0,0,0,0
2,53482,11050,36,13.65,1238631003,1277511093,658715225,17.0,9.0,15500.0,...,0,0,0,0,0,0,1,0,0,0
3,53483,15350,36,14.20,1136161107,1198371199,774316843,17.0,9.0,10100.0,...,0,0,0,0,0,0,1,0,0,0
4,53484,7800,36,10.00,1214960354,1214957804,832207268,17.0,9.0,13100.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20049,73529,10000,12,18.85,1130976000,1125964800,1018224000,6.0,3.0,3818.0,...,0,0,0,0,0,0,0,0,0,0
20050,73530,10000,12,29.30,1156204800,1157068800,1054425600,6.0,6.0,5502.0,...,1,0,0,0,0,0,0,0,0,0
20051,73531,11000,12,24.75,1144108800,1111622400,1037404800,8.0,3.0,4844.0,...,0,0,0,0,0,0,0,0,0,0
20052,73532,8000,12,22.00,1163808000,1116892800,1057017600,6.0,3.0,3495.0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
X_test.rename(columns={'index':'id'}, inplace=True)
print(X_test.shape)
X_test['label'] = predict_result
minmin = min(X_test['label'])
maxmax = max(X_test['label'])
X_test['label'] = X_test['label'].map(lambda x:(x-minmin)/(maxmax-minmin))
X_test[['id','label']].to_csv('%s_gh_v2.csv' % suffix, encoding='utf-8', index=False, mode='w')
!ls

(20054, 73)
202508260921_gh_v2.csv
bak
model_stacking.ipynb
process.ipynb
test.dat
test.dat.202508260921
train.dat
train.dat.202508260921


## 验证v2

In [44]:
from sklearn.model_selection import train_test_split

dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level"
dummy_fea = dummy_fea_str.split(',')

sample_data_0 = pd.read_csv(train_dat_path, engine = 'python');
sample_data = sample_data_0.fillna(0)

#1. lgb feature
#处理dummy变量
for _fea in dummy_fea:
    le = LabelEncoder()
    le.fit(sample_data[_fea].tolist())
    tmp = le.transform(sample_data[_fea].tolist())
    sample_data[_fea] = tmp
print('lgb sample data\n', sample_data)
sample_data_target = sample_data['label'].values
sample_data_x      = sample_data.drop(['label'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(sample_data_x, sample_data_target, test_size=0.2, random_state=None, shuffle=True)
lgb_dataset = Dataset(X_train=X_train, y_train=y_train, X_test = X_test, y_test= None, use_cache=False)


##2. xgb feature
sample_data = sample_data_0.fillna(0)
dummy_df = pd.get_dummies(sample_data.loc[:,dummy_fea])
print('xgb dummy_df\n ',dummy_df)
sample_data_concat = pd.concat([sample_data, dummy_df], axis=1)
sample_data_concat = sample_data_concat.fillna(0)
vaild_sample_data = sample_data_concat.drop(dummy_fea, axis=1)
print('xgb valid sample data', vaild_sample_data.columns)

sample_data_target = vaild_sample_data['label'].values
sample_data_x = vaild_sample_data.drop(['label'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(sample_data_x, sample_data_target, test_size=0.2, random_state=None, shuffle=True)
xgb_dataset = Dataset(X_train=X_train, y_train=y_train, X_test = X_test, y_test= None, use_cache=False)

model_xgb  = Regressor(dataset=xgb_dataset, estimator=xgb_feature,name='xgb',use_cache=False)
model_xgb2 = Regressor(dataset=xgb_dataset, estimator=xgb_feature2,name='xgb2',use_cache=False)
model_xgb3 = Regressor(dataset=xgb_dataset, estimator=xgb_feature3,name='xgb3',use_cache=False)
# model_lgb = Regressor(dataset=lgb_dataset, estimator=lgb_feature,name='lgb',use_cache=False)
model_gbdt = Regressor(dataset=xgb_dataset, estimator=gbdt_model,name='gbdt',use_cache=False)



lgb sample data
           id  title  career  zip_code  residence   loan  term  interest_rate  \
0          0      9       0       271          1   7200    36          10.95   
1          1      8      10       508          0  21300    36          12.95   
2          2      8       7       424          1  10400    60          21.05   
3          3      7       2       766          0  33050    36          16.40   
4          4      8       3        25          1   5200    36          14.35   
...      ...    ...     ...       ...        ...    ...   ...            ...   
53475  53475      2       2      1057          1   9000    12          23.55   
53476  53476      0      10       916          1   8000    12          30.70   
53477  53477      2      10      1043          1  10000    12           9.40   
53478  53478      0      10       960          2   9000    12          24.40   
53479  53479      0      10       976          1  10000    12          17.60   

       issue_time  syn

# 模型训练

In [45]:

pipeline = ModelsPipeline(model_xgb, model_xgb2, model_xgb3, model_gbdt)
stack_ds = pipeline.stack(k=5, seed=111, add_diff=False, full_test=True)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression, parameters={'fit_intercept': False})
predict_result = stacker.predict()

print('ModelsPipeline(model_xgb, model_xgb2, model_xgb3, model_gbdt) valid auc', roc_auc_score(y_test, predict_result))


[11:58:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:58:31] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:58:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false

## lgb feature

In [ ]:
train_data = pd.read_csv('train.dat',engine = 'python');
train_data = train_data.fillna(0)

test_data = pd.read_csv('test.dat',engine = 'python');
test_data = test_data.fillna(0)

train_test_data = pd.concat([train_data, test_data],axis=0,ignore_index = True)
train_test_data = train_test_data.fillna(0)

train_data = train_test_data.iloc[:train_data.shape[0],:]
test_data = train_test_data.iloc[train_data.shape[0]:,:]

#处理dummy变量
for _fea in dummy_fea:
    print(_fea)
    le = LabelEncoder()
    le.fit(train_data[_fea].tolist() + test_data[_fea].tolist())
    tmp = le.transform(train_data[_fea].tolist() + test_data[_fea].tolist())
    train_data[_fea] = tmp[:train_data.shape[0]]
    test_data[_fea]  = tmp[train_data.shape[0]:]
    
train_x = train_data.drop(['label'],axis=1)
test_x = test_data.drop(['label'],axis=1)
lgb_dataset = Dataset(train_x, train_data['label'], test_x, use_cache=False)

## xgb feature

In [ ]:
train_data = pd.read_csv('train.dat',engine = 'python');
train_data = train_data.fillna(0)

test_data = pd.read_csv('test.dat',engine = 'python');
test_data = test_data.fillna(0)

train_test_data = pd.concat([train_data, test_data], axis=0, ignore_index = True)
train_test_data = train_test_data.fillna(0)

dummy_df = pd.get_dummies(train_test_data.loc[:,dummy_fea])
train_test_data = pd.concat([train_test_data, dummy_df],axis=1)
train_test_data = train_test_data.drop(dummy_fea, axis=1)

train_train = train_test_data.iloc[:train_data.shape[0],:]
test_test = train_test_data.iloc[train_data.shape[0]:,:]

train_train_x = train_train.drop(['label'],axis=1)
test_test_x = test_test.drop(['label'],axis=1)

xgb_dataset = Dataset(X_train=train_train_x, y_train=train_train['target'], X_test = test_test_x, y_test= None, use_cache=False)

## stacking

In [ ]:
model_xgb = Regressor(dataset=xgb_dataset, estimator=xgb_feature,name='xgb',use_cache=False)
model_xgb2 = Regressor(dataset=xgb_dataset, estimator=xgb_feature2,name='xgb2',use_cache=False)
model_xgb3 = Regressor(dataset=xgb_dataset, estimator=xgb_feature3,name='xgb3',use_cache=False)
model_lgb = Regressor(dataset=lgb_dataset, estimator=lgb_feature,name='lgb',use_cache=False)
model_gbdt = Regressor(dataset=xgb_dataset, estimator=gbdt_model,name='gbdt',use_cache=False)
pipeline = ModelsPipeline(model_xgb, model_xgb2, model_xgb3, model_gbdt)
stack_ds = pipeline.stack(k=5, seed=111, add_diff=False, full_test=True)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression,parameters={'fit_intercept': False})
predict_result = stacker.predict()

In [ ]:
ans = pd.read_csv('testaa.csv')
ans['PROB'] = predict_result
minmin = min(ans['PROB']),
maxmax = max(ans['PROB'])
ans['PROB'] = ans['PROB'].map(lambda x:(x-minmin)/(maxmax-minmin))
ans['PROB'] = ans['PROB'].map(lambda x:'%.4f' % x)
ans.to_csv('./ans_stacking.csv',index=None)